## Final Project Submission

Please fill out:
* Student name: Cameryn
* Student pace: part time
* Scheduled project review date/time: 
* Instructor name: Victor Geislinger
* Blog post URL: 


# Importing important packages
For this project, the pandas, numpy, seaborn, matplotlib, and sqlite3 packages are needed and imported. The matplotlib is also declared as inline to avoid any potential issues in the future.

In [1]:
import pandas as pd # for dataframes
import numpy as np
import seaborn as sns # for making our charts more readable
import matplotlib.pyplot as plt
import sqlite3
from sqlalchemy import create_engine

%matplotlib inline

# Importing all data as CSV and TSV files

First, it is important to bring all of the data into a usable, readable format, in this case, pandas dataframes.

**Note that one file, 'rt.reviews.tsv' is encoded differently and thus needed to be converted using the correct encoding declaration.**

In [2]:
#Import data
bom_df = pd.read_csv('zippedData/bom.movie_gross.csv')
im_n_basics_df = pd.read_csv('zippedData/imdb.name.basics.csv')
im_akas_df = pd.read_csv('zippedData/imdb.title.akas.csv')
im_basics_df = pd.read_csv('zippedData/imdb.title.basics.csv')
im_crew_df = pd.read_csv('zippedData/imdb.title.crew.csv')
im_principals_df = pd.read_csv('zippedData/imdb.title.principals.csv')
im_ratings_df = pd.read_csv('zippedData/imdb.title.ratings.csv')
rt_movie_info_df = pd.read_csv('zippedData/rt.movie_info.tsv', sep='\t')
rt_reviews_df = pd.read_csv('zippedData/rt.reviews.tsv', sep='\t', encoding = 'latin1')
tmdb_movies_df = pd.read_csv('zippedData/tmdb.movies.csv')
tn_budgets_df = pd.read_csv('zippedData/tn.movie_budgets.csv')

After importing them, the .info() of each dataframe was checked in order to determine the data types of each column and determine what needed to be done in the below cells. That is not shown below for each dataframe, simply due to the amount of space that does take, but it is shown for the first dataframe, bom_df.

**Note the 'foreign_gross' column's type. This will prevent future statistical analysis, which is addressed in the future.**

In [3]:
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
title             3387 non-null object
studio            3382 non-null object
domestic_gross    3359 non-null float64
foreign_gross     2037 non-null object
year              3387 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


# Data Cleaning
In order to appropriately understand and interpret each dataframe in the context of the others, it is important to clean andappropriately format each column, removing unhelpful and ill-formated data and replacing where possible.

### Changing Column Type
The issue noted previously with bom_bf['foreign_gross'], is common across a few of our dataframes, thus the below function has been written to prevent unnecessary repetition.

In [4]:
# This Function strips unnecessary characters from data and converts it to numbers
def column_type_changer(column, df): 
    df[column] = df[column].str.replace('$','') # Removing non-numeric chars
    df[column] = df[column].str.replace(',','') # Removing non-numeric chars
    df[column] = df[column].str.replace(' minutes','') # Removing non-numeric chars
    df[column] = df[column].apply(pd.to_numeric) # Converts to numbers
    return df[column]

In [5]:
bom_df['foreign_gross'] = column_type_changer('foreign_gross', bom_df)

In [6]:
print(bom_df.info())
print(bom_df.describe())
bom_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
title             3387 non-null object
studio            3382 non-null object
domestic_gross    3359 non-null float64
foreign_gross     2037 non-null float64
year              3387 non-null int64
dtypes: float64(2), int64(1), object(2)
memory usage: 132.4+ KB
None
       domestic_gross  foreign_gross         year
count    3.359000e+03   2.037000e+03  3387.000000
mean     2.874585e+07   7.487281e+07  2013.958075
std      6.698250e+07   1.374106e+08     2.478141
min      1.000000e+02   6.000000e+02  2010.000000
25%      1.200000e+05   3.700000e+06  2012.000000
50%      1.400000e+06   1.870000e+07  2014.000000
75%      2.790000e+07   7.490000e+07  2016.000000
max      9.367000e+08   9.605000e+08  2018.000000


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000.0,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010
3,Inception,WB,292600000.0,535700000.0,2010
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010


Note the number of null objects in the 'foreign_gross' column. This could be attributed to a number of things, though often it could simply be that the film wasn't released abroad. Without context, it may cause more problems that it would solve to replace that information. Further exploration of the other data may be necessary.

In [7]:
im_n_basics_df.info()
im_n_basics_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
nconst                606648 non-null object
primary_name          606648 non-null object
birth_year            82736 non-null float64
death_year            6783 non-null float64
primary_profession    555308 non-null object
known_for_titles      576444 non-null object
dtypes: float64(2), object(4)
memory usage: 27.8+ MB


,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [8]:
im_akas_df.info()
im_akas_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331703 entries, 0 to 331702
Data columns (total 8 columns):
title_id             331703 non-null object
ordering             331703 non-null int64
title                331703 non-null object
region               278410 non-null object
language             41715 non-null object
types                168447 non-null object
attributes           14925 non-null object
is_original_title    331678 non-null float64
dtypes: float64(1), int64(1), object(6)
memory usage: 20.2+ MB


,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0
1,tt0369610,11,Jurashikku warudo,JP,NaN,imdbDisplay,NaN,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,NaN,imdbDisplay,NaN,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,NaN,NaN,short title,0.0
4,tt0369610,14,Jurassic World,FR,NaN,imdbDisplay,NaN,0.0


In [9]:
im_basics_df.info()
im_basics_df.head()
im_basics_df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
tconst             146144 non-null object
primary_title      146144 non-null object
original_title     146123 non-null object
start_year         146144 non-null int64
runtime_minutes    114405 non-null float64
genres             140736 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


Index(['tconst', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres'],
      dtype='object')

In [10]:
im_crew_df.info()
im_crew_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 3 columns):
tconst       146144 non-null object
directors    140417 non-null object
writers      110261 non-null object
dtypes: object(3)
memory usage: 3.3+ MB


,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


In [11]:
im_principals_df.info()
im_principals_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028186 entries, 0 to 1028185
Data columns (total 6 columns):
tconst        1028186 non-null object
ordering      1028186 non-null int64
nconst        1028186 non-null object
category      1028186 non-null object
job           177684 non-null object
characters    393360 non-null object
dtypes: int64(1), object(5)
memory usage: 47.1+ MB


,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


In [12]:
im_ratings_df.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [13]:
rt_movie_info_df.info()
rt_movie_info_df['box_office'] = column_type_changer('box_office',rt_movie_info_df)
rt_movie_info_df['runtime'] = column_type_changer('runtime',rt_movie_info_df)
rt_movie_info_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
id              1560 non-null int64
synopsis        1498 non-null object
rating          1557 non-null object
genre           1552 non-null object
director        1361 non-null object
writer          1111 non-null object
theater_date    1201 non-null object
dvd_date        1201 non-null object
currency        340 non-null object
box_office      340 non-null object
runtime         1530 non-null object
studio          494 non-null object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104.0,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,600000.0,108.0,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116.0,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128.0,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200.0,NaN


In [14]:
rt_reviews_df.info() #use this data to potentially invite specific reviewers
rt_reviews_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
id            54432 non-null int64
review        48869 non-null object
rating        40915 non-null object
fresh         54432 non-null object
critic        51710 non-null object
top_critic    54432 non-null int64
publisher     54123 non-null object
date          54432 non-null object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [15]:
tmdb_movies_df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [16]:
tn_budgets_df['production_budget'] = column_type_changer('production_budget', tn_budgets_df)
tn_budgets_df['domestic_gross'] = column_type_changer('domestic_gross', tn_budgets_df)
tn_budgets_df['worldwide_gross'] = column_type_changer('worldwide_gross', tn_budgets_df)
tn_budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


# Creating a Database
It is necessary to create a database first to tie all of the previous data together and to allow for queries.

In [17]:
# Create the movies database
conn = sqlite3.connect('movies.sqlite')
cur = conn.cursor()


In [18]:
cur.execute("""CREATE TABLE movies (
                                   tconst PRIMARY KEY, 
                                   primary_title TEXT, 
                                   original_title TEXT, 
                                   start_year INTEGER,
                                   runtime_minutes INTEGER, 
                                   genres TEXT
                                   
);""")

OperationalError: table movies already exists

In [ ]:
engine = create_engine('sqlite://', echo=False)
im_basics_df.to_sql('movies', con=engine)

engine.execute("""SELECT * 
                from movies;""").fetchall()